# Implementação básica do algoritmo k-means

Em grande parte baseada na implementação disponível em [dataquestio/kmeans](https://github.com/dataquestio/project-walkthroughs/tree/master/kmeans)

## Imports e Variáveis Globais

In [9]:
import numpy as np
import pandas as pd

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from IPython.display import clear_output



PLOT_RESULTS = True
DEBUG = True

N = 100000
D = 5

K = 3
MAX_ITERATIONS = 500

## Criando o Dataset

- `N` datapoints
- Datapoints `D`-dimensionais
- Floats randomizados ➡ intervalo de [1, 10) (ou [1, 10], dependendo da método de arredondamento de float usado. Isto não deve ter nenhuma relevância estatística, no entanto)

In [10]:
dataset  = [[np.random.uniform(1, 10) for _ in range(D)] for _ in range(N)]
dataset = pd.DataFrame(dataset, columns=[f'd{i}' for i in range(D)])

# # # ! Descomente para exibir
# dataset

## K-Means (CPU)

In [11]:
def kMeansCPU(dataset:pd.DataFrame, k=3, maxIter=100, plotResults=False, debug=False):
    if plotResults:
        # Inicializando variáveis para exibição gráfica
        pca = PCA(n_components=2) # dois eixos no gráfico
        dataset_2D = pca.fit_transform(dataset)

    # Gerando centróides iniciais randomicamente
    centroids = pd.concat([(dataset.apply(lambda x: float(x.sample().iloc[0]))) for _ in range(k)], axis=1) # * Paralelizar isto provavelmente é irrelevante, visto que sempre teremos poucos centróides
    centroids_OLD = pd.DataFrame()

    iteration = 1

    while iteration <= maxIter and not centroids_OLD.equals(centroids):
        if plotResults or debug: clear_output(wait=True)
        if debug: debugStr = f'Iteration {iteration}\n\nCentroids:\n{centroids.T}\n\n'

        # Para cada datapoint, calcular distâncias entre ele e cada centróide; depois, encontrar o centróide mais próximo e salvar seu index
        distances = centroids.apply(lambda x: np.sqrt(((dataset - x) ** 2).sum(axis=1))) # ! Parte altamente paralelizável!
        if debug: debugStr += f'Distances:\n{distances}\n\n'
        closestCent = distances.idxmin(axis=1)
        del distances
        if debug: debugStr += f'Closest centroid index:\n{closestCent}\n\n'

        centroids_OLD = centroids
        centroids = dataset.groupby(closestCent).apply(lambda x: np.exp(np.log(x).mean())).T # ! Parte altamente paralelizável!

        if plotResults:
            # Plotando clusters
            centroids_2D = pca.transform(centroids.T)
            plt.title(f'Iteration {iteration}')
            plt.scatter(x=dataset_2D[:,0], y=dataset_2D[:,1], c=closestCent)
            plt.scatter(x=centroids_2D[:,0], y=centroids_2D[:,1], marker='+', linewidths=2, color='red')
            plt.show()

        if debug: print(debugStr)

        iteration += 1

    return closestCent

In [12]:
# # # ! Descomente para rodar de fato
# result = kMeansCPU(dataset, K, MAX_ITERATIONS)

# result

# Testando com um Dataset Real (Pequeno) — *Iris*

Vamos agora testar esta implementação do k-means usando um dataset real de tamanho bem trivial.

Utilizaremos o [*Iris* Data Set](https://archive.ics.uci.edu/ml/datasets/Iris), que consiste de dados a respeito de espécimes de [flores do gênero Íris](https://pt.wikipedia.org/wiki/Iris_(g%C3%A9nero)), popularmente chamadas de **Lírios**.

## Dataset

O *Iris* consiste em **150 instâncias**, com dados multivariados de **quatro atributos**:

1. Comprimento da sépala em centímetros
2. Largura da sépala em centímetros
3. Comprimento da pétala em centímetros
4. Largura da pétala em centímetros

Há também um quinto atributo de **Classe**, o **tipo da planta**, que é o que esperamos predizer utilizando o algoritmo. Há **três classes** neste dataset:

- Iris Setosa
- Iris Versicolour
- Iris Virginica

Assim, temos nossas variáveis para execução do k-means:

- `N` = 150
- `D` = 4
- `K` = 3

### Carregando o Dataset

In [13]:
# Imports
from os.path import exists as os_path_exists
from urllib.request import urlopen



# Novas variáveis globais
MAX_ITERATIONS = 100
PLOT_RESULTS = True
DEBUG = True
# ? K será inferido pela quantidade de classes no dataset
# // K = 3



# Função auxiliar
def downloadFileIfNeeded(filePath, url):
    if not os_path_exists(filePath):
        with urlopen(url) as f:
            html = f.read().decode('utf-8')
        with open(filePath, 'w') as f:
            f.write(html)



datasetUrl = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
datasetFilePath = './iris.data'
columnNames = ['sepalLen', 'sepalWid', 'petalLen', 'petalWid', 'class']

# Baixando dataset diretamente da internet, se necessário
downloadFileIfNeeded(datasetFilePath, datasetUrl)

# Lendo dataset do arquivo
with open(datasetFilePath, 'r') as datasetFile:
    dataset = pd.read_csv(datasetFilePath, names=columnNames)

# # # ! Descomente para exibir
# print(dataset)

### Preparando o Dataset

In [14]:
# Transformando a coluna "class" em números (0, 1, …)
classFactorized = dataset['class'].factorize()
# Salvando os nomes de classe na ordem usada pelo .factorize()
classes = list(classFactorized[1])
# // # Sobrescrevendo a coluna do dataset pelos indexes obtidos
# // dataset['class'] = classFactorized[0]

# # # ! Descomente para exibir
# print(f'##### Dataset #####\n{dataset}\n\n')

# Ignorando a coluna "class"
datasetTreated = dataset.drop(['class'], axis=1)
# Normalizando o dataset (normalização min-max), para que todos valores estejam no intervalo [1, 10]
datasetTreated = ((datasetTreated - datasetTreated.min()) / (datasetTreated.max() - datasetTreated.min())) * 9 + 1

# # # ! Descomente para exibir
# print(f'##### Dataset (tratado e normalizado, intervalo [1, 10]) #####\n{datasetTreated}')

# Inferindo número de grupos pela quantidade de classes no dataset
K = len(classes)

## Rodando o K-Means

In [15]:
# # # ! Descomente para rodar de fato
# results = kMeansCPU(datasetTreated, K, MAX_ITERATIONS, PLOT_RESULTS, DEBUG)

## Conferindo Resultados

Com a tabela `ClosestCentroids` em mãos, temos a classificação de cada instância do dataset *Iris* e, assim, podemos conferir os resultados obtidos através do k-means.

Para isto, basta comparar a segunda coluna da tabela devolvida pelo `kMeansCPU()` com a última coluna do dataset Iris, que informa a classe correta de cada instância.

Essa comparação, no entanto, não é tão simplória de se realizar. Não há como sabermos qual o mapeamento correto entre a classificação feita pelo k-means e as classes reais das instâncias do dataset. Isto é, não há como discernir se o `1` na coluna de classificação de centróides se refere a uma identificação de uma `Iris-versicolor` ao invés de uma `Iris-virginica`, por exemplo.

Por isso, temos que realizar o **maximum matching** atingido pelo resultado do k-means, para realizar uma avaliação justa de seu desempenho.

### Maximum Matching

Teremos que avaliar todas as possibilidades de *mapeamento centróide–classe* e considerar apenas aquela que gera o melhor resultado.

Para isso, é necessário calcular os acertos do resultado diversas vezes. Como há $ k! $ maneiras de se interpretar `K` centróides, isto irá gerar um **custo computacional de $ n \cdot k! $ operações de comparação** entre classificações.

Podemos reduzir este tempo de computação salvando e reaproveitando inteligentemente a quantidade de acertos de classificação computados para cada *mapeamento centróide–classe*.

Por exemplo, há **duas** interpretações dos `K` centróides que possuem um mapeamento `0 -> Iris-Setosa`. São eles: `[Iris-setosa, Iris-versicolor, Iris-virginica]` e `[Iris-setosa, Iris-virginica, Iris-versicolor]`. Ao contar os acertos da primeira destas permutações dos `K` centróides, vamos obter os acertos de linhas do resultado do k-means onde o centróide mais próximo é o de index `0`, e cuja interpreção atual é de este `0` indicar uma flor `Iris-Setosa`. Como iremos usar essa mesma interpretação para centróides de index `0` na próxima permutação, podemos pular essa contagem redundante utilizando o resultado obtido na contagem anterior.

Utilizando este método, conseguimos **reduzir o custo de computação** de acertos de classificação **para $ n^2 $ operações de comparação**!

In [16]:
from itertools import permutations

# # # ! Descomente para exibir
# print(f'result:\n{results}\n')
# print(f'dataset:\n{dataset}\n')
# print(f'classes:\n{classes}\n')

def getClassificationHits(results:pd.DataFrame, dataset:pd.DataFrame, classColumn:str|int='class', classes:list[str]=None, debug=False):
    '''Função auxiliar que retorna uma tupla com três informações: (1) a quantidade de acertos de classificação expressos no dataframe `results`; (2) as quantidades destes acertos separadas por classes; e (3) a interpretação das classes usadas para encontrar o melhor resultado
    
    Para determinar acertos e erros, é usada a coluna de nome/index `classColumn` do dataframe `dataset` como fonte de verdade. O maximum matching é feito para encontrar a melhor interpretação dos resultados e usá-la como resultado final

    `results` deve ser um dataframe com duas colunas, a primeira sendo o index do datapoint e a segunda sendo o index da classe à qual o datapoint foi classificado
    
    Uma lista com os nomes das classes pode ser passada em `classes`, para agilizar o processo. Se nada for passado, as classes serão inferidas pela coluna de nome/index `classColumn` do dataframe `dataset`

    Exemplo de retorno: `(118, {'Iris-setosa': 35, 'Iris-versicolor': 39, 'Iris-virginica': 44}, ('Iris-virginica', 'Iris-setosa', 'Iris-versicolor'))`
    
    O retorno acima significa que (1) houveram `118` acertos totais, (2) sendo `35` desses da classe `Iris-setosa`, `39` da classe `Iris-versicolor` e `44` da classe `Iris-virginica` e (3) que a interpretação usada para obter este melhor resultado foi: classe `0` em `results` = `Iris-virginica`, classe `1` = `Iris-setosa` e  classe `2` = `Iris-versicolor`.'''

    def getHitsForClassAndPosition(class_:str, position:int, results:pd.DataFrame=results, dataset:pd.DataFrame=dataset, classColumn:str|int=classColumn):
        '''Função auxiliar interna que calcula os acertos de uma única interpretação de uma única classe
        
        Tal interpretação é expressa em duas variáveis: `class_` -> nome da classe (assim como está expressa na coluna `classColumn` do `dataset`) e `position` -> posição da classe quanto ao array das possíveis k classes ([0, …, k-1] da segunda coluna de `results`)
        
        Exemplo:
        
        `getHitsForClassAndPosition('Iris-setosa', 1)` irá calcular os acertos em `results` ao considerar a classificação de index `1` como simbolizando a classe `Iris-setosa`'''

        hits = 0

        print(f'Counting hits for class "{class_}" being index "{position}" in the results dataset...')

        for rowIndex in range(0, len(results)):
            row = results.iloc[[rowIndex]]
            # datapointIndex = row.index.values[0]
            resultClassIndex = row.values[0]

            # isCorrect = None
            # correctClass = None

            # Conferimos apenas linhas onde o index em "results" é o mesmo em "position", pois só nos importamos com acertos para palpites deste index
            if resultClassIndex == position:
                # Obtendo a classe correta da nossa "fonte de verdade", o dataset "dataset"
                correctClass = dataset.iloc[[rowIndex]][classColumn].values[0]

                # Checando se foi um acerto
                isCorrect = True if str(correctClass) == str(class_) else False
                if isCorrect: hits += 1
                
            # print(f'dpIndex = {datapointIndex}; result = {resultClassIndex}; rowDataset = {correctClass}; isCorrect = {isCorrect}')

        print(f'Total hits: {hits}\n')
        return hits
        return np.random.randint(0, 50 + 1)

    if debug: print('#################### Computing classification hits... ####################\n')

    if classes is None or len(classes) == 0:
        classes = list(dataset[classColumn].factorize()[1])
        if debug: print(f'classes (inferred from `dataset`):\n{classes}\n')

    # Foçar classes para strings
    classes = [str(c) for c in classes]

    # Encontrar todas as permutações possíveis das classes. A posição de uma classe na permutação será usada para atrelar uma classe ao centroid de mesmo index. Por exemplo, na permutação ('Iris-versicolor', 'Iris-setosa', 'Iris-virginica'), o centróide 0 corresponderá à classe 'Iris-versicolor', o centróide 1 à classe 'Iris-setosa' e o centróide 2 à classse 'Iris-virginica'. Cada datapoint terá sua classificação definida através dessa relação
    classPerm = list(permutations(classes))

    if debug:
        print(f'classesPermutatons (len={len(classPerm)}):\n')
        for perm in classPerm: print(perm)
        print('\n')

    # Dicionário auxiliar que guarda a quantidade de acertos por classe e posição na lista de classes. Isto economizará poder computacional, pois não repetiremos cálculos redundantes de hits de uma classe numa mesma posição.
    # Estrutura:
    # {
    #     'Iris-setosa/0': 50,
    #     'Iris-setosa/1': 0,
    #     '<class>/<positionInPermutation>': 45,…
    # }
    hitsByclassAndPosition = {}

    bestHits = -1
    bestHitsPerClass = None
    bestPerm = None

    # Para cada permutação das classes
    for permutation in classPerm:
        # print(f'permutation = {permutation}\n')
        hitsPerClass = dict.fromkeys(classes, -1)

        # Para cada classe na permutação atual
        for classPosition, class_ in enumerate(permutation):
            # Se os hits para essa classe nessa posição da permutação já foram computados, vamos usá-los. Se não, os computamos pela primeira vez e salvamos para usos posteriores
            classAndPositionStr = f'{class_}/{classPosition}' # Chave a ser usada no dicionário
            hits:int = hitsByclassAndPosition.get(classAndPositionStr, None)
            if hits is None:
                # TODO
                # TODO
                # TODO: Implementar o cálculo real dos acertos aqui
                # TODO
                # TODO
                hits = getHitsForClassAndPosition(class_, classPosition)
                hitsByclassAndPosition[classAndPositionStr] = hits

            # Salvando a quantidade de acertos na classe atual, seja qual for
            hitsPerClass[class_] = hits
            # print(f'classPosition = {classPosition}; class_ = {class_}\nhitsPerClass = {hitsPerClass}\nhitsByclassAndPosition = {hitsByclassAndPosition}\n')

        # Computar acertos totais dessa permutação
        totalHits = sum(hitsPerClass.values())

        if totalHits > bestHits:
            bestHits = totalHits
            bestHitsPerClass = hitsPerClass
            bestPerm = permutation

        print(f'totalHits = {totalHits}\nbestHits = {bestHits}; bestHitsPerClass = {bestHitsPerClass}; bestPerm = {bestPerm}\n\n')

    return (bestHits, bestHitsPerClass, bestPerm)



# # # ! Descomente para rodar de fato
# hits, hitsPerClass, centroidClasses  = getClassificationHits(results, dataset, classes=classes, debug=DEBUG)

# if DEBUG: print(f'Hits: {hits}\nHits per class: {hitsPerClass}\nCentroid classes: {centroidClasses}\n')

### Resultados

Rodando a função **`getClassificationHits()`** nos resultados do k-means, encontramos resultados bem esperados:

- Acertos totais entre **130–140**, de 150 classificações (**86,66–93,33%** de acerto)
- Acertos por classe:
  - `Iris-setosa`: quase sempre **50** de 50 (**100%**)
  - `Iris-versicolor`: **35–38** de 50 (**70–76%**)
  - `Iris-virginica`: **45–48** de 50 (**90–96%**)

Estes resultados são exatamente o que esperamos do dataset **Iris**, já que nele temos uma classe linearmente separável (`Iris-setosa`) e duas que NÃO são linearmente separáveis uma da outra (`Iris-versicolor` e `Iris-virginica`).